### Hyperparameter searching with GridSearch

In [117]:
import pandas as pd
from numpy import random
from keras.activations import relu, elu
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

The dataset includes data from 768 women with 8 characteristics, in particular:

    Number of times pregnant
    Plasma glucose concentration a 2 hours in an oral glucose tolerance test
    Diastolic blood pressure (mm Hg)
    Triceps skin fold thickness (mm)
    2-Hour serum insulin (mu U/ml)
    Body mass index (weight in kg/(height in m)^2)
    Diabetes pedigree function
    Age (years)

The last column of the dataset indicates if the person has been diagnosed with diabetes (1) or not (0)

In [120]:
df  = pd.read_csv("e:/$Notebooks/pima-indians-diabetes.csv", header = 0)

X = df.iloc[:,0:8]
y = df.iloc[:,8]

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [127]:
def model_diabetes(optimizer='Adam', init='uniform', activation='relu', dropout = 0.5, layer_size=16):
    
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation=activation,kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(layer_size, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation=activation))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [128]:
param_grid = {
    'layer_size':[24,64],
    'activation': ['relu', 'elu'],
    'optimizer' : ['Adam', 'Adamax'],
    'batch_size': [10, 20, 30],
    'dropout'   : [0.5,0.4,0.3]
}

In [129]:
# fix random seed for reproducibility
seed = 7
random.seed(seed)

# create model
#model = KerasClassifier(build_fn=create_model, verbose=0)

# grid search epochs, batch size and optimizer ('TFOptimizer' throws error)
optimizers = ['SGD','RMSprop','Adagrad','Adadelta','Adam','Adamax','Nadam']     
# possible activations 'softmax','softplus','softsign','relu','tanh','sigmoid','hard_sigmoid','linear'
# activations = ['softmax','softplus','softsign','relu','tanh','sigmoid','hard_sigmoid','linear']
activations = ['softmax','softplus','softsign','relu','tanh','sigmoid','hard_sigmoid','linear']

# 'glorot_uniform', 'normal', 'uniform'
# init = ['uniform','lecun_uniform','normal','identity','orthogonal','zero','one','glorot_normal','glorot_uniform', 'he_normal', 'he_uniform']
# init = ['uniform','lecun_uniform','normal','orthogonal','zero','one','glorot_normal','glorot_uniform', 'he_normal', 'he_uniform']
init = ['normal']
        
# just one epoch to save computational time (300 is optimum for this 12,8,1 model)
epochs = [100]

# batches can be 1,3,5,10,20,40
batches = [5]

In [ ]:
m = KerasClassifier(build_fn=model_diabetes, verbose=0)  # to use it with scikit

#param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init, activation=activations)


grid = GridSearchCV(cv=3, estimator=m, param_grid=param_grid, n_jobs=1, verbose=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

results = pd.DataFrame({'Mean':means[:],'std':stds[:],'parameters':params[:]})
results = results.sort_values(by=['Mean'],ascending=[0])
print(tabulate(results, headers='keys', tablefmt='psql'))

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [96]:
param_grid = {
    'layer_size':[128,256,512],
    'activation': ['elu','relu'],
    'optimizer' : ['Adam'],
    'batch_size': [5,10,20]
}

In [97]:
grid = GridSearchCV(cv=3, estimator=m, param_grid=param_grid, n_jobs=1, verbose=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

results = pd.DataFrame({'Mean':means[:],'std':stds[:],'parameters':params[:]})
results = results.sort_values(by=['Mean'],ascending=[0])
print(tabulate(results, headers='keys', tablefmt='psql'))

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 75.4min finished


Best: 0.764323 using {'activation': 'relu', 'batch_size': 5, 'layer_size': 128, 'optimizer': 'Adam'}
0.750000 (0.036782) with: {'activation': 'elu', 'batch_size': 5, 'layer_size': 128, 'optimizer': 'Adam'}
0.748698 (0.025780) with: {'activation': 'elu', 'batch_size': 5, 'layer_size': 256, 'optimizer': 'Adam'}
0.750000 (0.033299) with: {'activation': 'elu', 'batch_size': 5, 'layer_size': 512, 'optimizer': 'Adam'}
0.675781 (0.025516) with: {'activation': 'elu', 'batch_size': 10, 'layer_size': 128, 'optimizer': 'Adam'}
0.694010 (0.060711) with: {'activation': 'elu', 'batch_size': 10, 'layer_size': 256, 'optimizer': 'Adam'}
0.703125 (0.025315) with: {'activation': 'elu', 'batch_size': 10, 'layer_size': 512, 'optimizer': 'Adam'}
0.652344 (0.025315) with: {'activation': 'elu', 'batch_size': 20, 'layer_size': 128, 'optimizer': 'Adam'}
0.651042 (0.024774) with: {'activation': 'elu', 'batch_size': 20, 'layer_size': 256, 'optimizer': 'Adam'}
0.675781 (0.042192) with: {'activation': 'elu', 'batch